In [1]:
import requests
import json
import pandas as pd
import time

In [2]:
#Retrieving a list of account ID's based on first three letters of player's nickname

In [3]:
three_letters = ['nic', 'das', 'jee', 'kvi', 'ddt', 'tec', 'ein', 'pek', 'zed', 'bot', 'maw', 'egl', 
                       'kej', 'cab', 'vyb', 'abc', 'fik', 'hit', 'ill', 'lak', 'siu', 'ulj']

In [4]:
base_url = "https://api.worldoftanks.eu/wot/account/list/?application_id="
application_id = '67369ded9212f123394343e566e'

player_list = []

for i in three_letters:
    response = requests.get(base_url + application_id + '&type=startswith&search=' + i)
    iload = response.json()
    player_list.extend(iload['data'])    

In [5]:
account_ids_str = [str(d['account_id']) for d in player_list]

In [6]:
#Splitting account id's to 100 per chunk and joining them to use in request URL's
#(API allows only 100 id's per one request)

In [7]:
two_d_list = [account_ids_str[x:x + 100] for x in range(0, len(account_ids_str), 100)]

In [8]:
string_chunk = ['%2C'.join(i) for i in two_d_list]

In [9]:
string_chunk

['501690965%2C500164554%2C500939483%2C506010227%2C504226953%2C514407049%2C629984633%2C509122694%2C558909212%2C503273745%2C552609954%2C543855283%2C508791795%2C601500117%2C518534707%2C501758275%2C681588374%2C680043160%2C538919880%2C510781051%2C517119648%2C591785930%2C501711030%2C514003309%2C526005877%2C576092602%2C520033343%2C501137466%2C542711078%2C509533363%2C503145898%2C603976681%2C635754841%2C518234108%2C580465131%2C527655744%2C620421127%2C508960098%2C507510098%2C546071927%2C599102210%2C501960879%2C518829053%2C595739205%2C535998954%2C516979147%2C502234188%2C502808234%2C509017259%2C628124098%2C505032003%2C503425012%2C510228993%2C538382491%2C537146521%2C522978482%2C507804411%2C501443902%2C516705902%2C676346816%2C504459939%2C509547999%2C553319674%2C507747782%2C518814271%2C513858620%2C509924844%2C507427565%2C504545668%2C501760002%2C504609842%2C604542737%2C518156471%2C506420193%2C512697512%2C548364823%2C502284000%2C502907494%2C511986657%2C528193693%2C553245349%2C510463728%2C510613471%2C50

In [10]:
#Now getting the player data and storing in JSON

In [11]:
player_data = []

In [12]:
url = "https://api.worldoftanks.eu/wot/account/info/?application_id="
application_id = '67369ded9212f123394343e566ee'
params = '&fields=account_id%2C+created_at%2C+global_rating%2C+statistics.all.battles%2C+statistics.all.max_damage%2C+statistics.all.max_damage_tank_id%2C+statistics.all.max_frags%2C+statistics.all.max_frags_tank_id%2C+statistics.all.max_xp%2C+statistics.all.max_xp_tank_id%2C+statistics.all.shots%2C+statistics.all.wins&account_id='
for i in string_chunk:
    response1 = requests.get(url + application_id + params + i)
    item_load = response1.json()
    player_data.append(item_load)    
    
    time.sleep(1.5)

In [13]:
# Flattening the JSON

In [ ]:
pre_flattened = {}
for i in range(len(player_data)):
    pre_flattened.update(player_data[i])

In [ ]:
flattened = []
for acc_id in account_ids_str:
    flattened.append(pre_flattened[acc_id])

In [17]:
#Exploring and cleaning data

In [18]:
data = pd.json_normalize(flattened)
data.to_csv('Player-data.csv', sep = ',', index = False)

In [2]:
df = pd.read_csv('Player-data.csv' , index_col = 'account_id')
df_copy = df.copy()

In [3]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2189 entries, 501690965 to 512168399
Data columns (total 11 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   created_at                         2189 non-null   int64  
 1   global_rating                      2189 non-null   int64  
 2   statistics.all.max_frags_tank_id   1348 non-null   float64
 3   statistics.all.max_xp_tank_id      1512 non-null   float64
 4   statistics.all.max_xp              2189 non-null   int64  
 5   statistics.all.wins                2189 non-null   int64  
 6   statistics.all.battles             2189 non-null   int64  
 7   statistics.all.max_frags           2189 non-null   int64  
 8   statistics.all.shots               2189 non-null   int64  
 9   statistics.all.max_damage_tank_id  1233 non-null   float64
 10  statistics.all.max_damage          2189 non-null   int64  
dtypes: float64(3), int64(8)
memory usage: 205.2 KB


In [21]:
df_copy.describe()

,created_at,global_rating,statistics.all.max_frags_tank_id,statistics.all.max_xp_tank_id,statistics.all.max_xp,statistics.all.wins,statistics.all.battles,statistics.all.max_frags,statistics.all.shots,statistics.all.max_damage_tank_id,statistics.all.max_damage
count,2.189000e+03,2189.000000,1348.000000,1512.000000,2189.000000,2189.000000,2189.000000,2189.000000,2189.000000,1233.000000,2189.000000
mean,1.444916e+09,1295.990863,10087.967359,12641.380952,688.166286,1127.813157,2295.124258,2.959799,19585.692554,13958.242498,1166.532663
std,1.142468e+08,1629.593545,16017.760983,18832.663651,771.278617,3745.622515,7467.640606,3.142012,64811.979603,18772.691012,1999.043658
min,1.281652e+09,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,1.359881e+09,0.000000,1809.000000,1553.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2577.000000,0.000000
50%,1.392888e+09,745.000000,3329.000000,3361.000000,402.000000,8.000000,16.000000,2.000000,186.000000,4945.000000,165.000000
75%,1.526854e+09,1900.000000,8257.000000,11777.000000,1157.000000,178.000000,363.000000,5.000000,3946.000000,15361.000000,1420.000000
max,1.702195e+09,10079.000000,64817.000000,65073.000000,3366.000000,43106.000000,79794.000000,13.000000,938634.000000,64817.000000,11568.000000


In [22]:
pd.isna(df_copy).sum()

created_at                             0
global_rating                          0
statistics.all.max_frags_tank_id     841
statistics.all.max_xp_tank_id        677
statistics.all.max_xp                  0
statistics.all.wins                    0
statistics.all.battles                 0
statistics.all.max_frags               0
statistics.all.shots                   0
statistics.all.max_damage_tank_id    956
statistics.all.max_damage              0
dtype: int64

In [4]:
df_copy.columns = df_copy.columns.str.removeprefix('statistics.all.').to_list()

In [5]:
df_copy['created_at'] = pd.to_datetime(df_copy['created_at'], unit = 's').dt.date

In [6]:
df_copy.fillna(0, inplace=True)

In [7]:
df_copy.to_csv('Player-Data-clean.csv', sep= ',')

In [29]:
#Retrieving Player Vehicle data

In [130]:
vehicle_data = {}

In [131]:
url = "https://api.worldoftanks.eu/wot/account/tanks/?application_id="
application_id = '67369ded9212f123394343e566ee'

for i in string_chunk:
    response1 = requests.get(url + application_id + '&account_id=' + i)
    item_load = response1.json()
    vehicle_data.update(item_load['data'])    
    
    time.sleep(1.5)

In [136]:
with open("Vehicle-data.json", "w") as outfile:
    json.dump(vehicle_data, outfile, indent = 2)

In [ ]:
#Flattening JSON

In [2]:
with open("Vehicle-data.json", "r") as file:
    data = json.load(file)

In [22]:
list_of_keys = [i for i in data]

['510228993',
 '501760002',
 '620421127',
 '548364823',
 '507747782',
 '513996845',
 '535672881',
 '504609842',
 '518534707',
 '501137466',
 '513858620',
 '518814271',
 '595739205',
 '502234188',
 '501690965',
 '652479581',
 '502907494',
 '503273745',
 '516705902',
 '602187890',
 '506010227',
 '526005877',
 '510781051',
 '509122694',
 '504226953',
 '504183955',
 '681588374',
 '507467927',
 '680043160',
 '537146521',
 '538382491',
 '528193693',
 '517119648',
 '552609954',
 '504459939',
 '553245349',
 '512697512',
 '502808234',
 '509017259',
 '501960879',
 '522978482',
 '543855283',
 '501711030',
 '518156471',
 '577923771',
 '591785930',
 '509547999',
 '500939483',
 '502284000',
 '518632162',
 '507427565',
 '510463728',
 '553319674',
 '507804411',
 '599102210',
 '500689671',
 '510781370',
 '604542737',
 '558909212',
 '542711078',
 '538811183',
 '514407049',
 '587898335',
 '501443902',
 '527655744',
 '505032003',
 '502788933',
 '511986657',
 '507510098',
 '523444565',
 '635754841',
 '5089

In [103]:
df=[]
keys = []
for i in list_of_keys:
    for j in range(len(data[i])):
        df.append(data[i][j])
        keys.append(str(i))

In [104]:
clean_df = pd.json_normalize(df)
clean_df['account_keys'] = keys

In [105]:
clean_df.set_index('account_keys', inplace = True)

In [107]:
clean_df.to_csv('Playe-Vehicle-data-clean.csv', sep = ',')